In [3]:
from tensorflow.keras import layers
from tensorflow.keras.applications import vgg16
from tensorflow.keras.utils import plot_model, image_dataset_from_directory
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow import convert_to_tensor
import tensorflow.keras.backend as K

import tensorflow as tf

tf.compat.v1.enable_eager_execution()
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
import pandas as pd
import numpy as np
import pathlib
import cv2
import os

### Paths

In [5]:
images_path = '../../../Dataset/img_celeba_cropped'
identity_path = '../../../Dataset/identity_CelebA.txt'
attributes_path = '../../../Dataset/list_attr_celeba.txt'

### Hyperparameters

In [6]:
images_num = 50000
image_shape = (224, 224, 3)

batch_size = 32
lr=0.001

### Creating pairs

In [7]:
celeb_identity = pd.read_csv(identity_path, sep = " ", names=["image", "identity"])[:images_num-1]
celeb_identity["identity"] = celeb_identity["identity"].astype("string")

celeb_attrs = pd.read_csv(attributes_path, sep = "\s+")[:images_num-1]

In [8]:
def get_balanced_pair(column:str, pairing_column:str, df=celeb_identity):
    # getting random identity from the provided column to use for balanced pair
    random_id = df[column].sample(1, replace=True).to_string(index=False)
    containing_id_list = df.loc[df[column] == random_id][pairing_column].sample(2, replace=True).to_list()
    
    # random pictures for pair generation
    df_without_id = df.loc[df[pairing_column] != random_id]
    negative = df[pairing_column].sample(1, replace=True).to_list()
    
    return [[containing_id_list[0], containing_id_list[1], 1], [containing_id_list[0], negative[0], 0]]

In [9]:
balanced_pairs = []

for record in range(100):
    pair = get_balanced_pair("identity", "image")
    balanced_pairs.append(pair[0])
    balanced_pairs.append(pair[1])
    
df = pd.DataFrame(balanced_pairs, columns =['left', 'right', 'label'])
df['left'] = images_path + '/' + df['left']
df['right'] = images_path + '/' + df['right']

In [10]:
df.head()

,left,right,label
0,../../../Dataset/img_celeba_cropped/021509.jpg,../../../Dataset/img_celeba_cropped/042762.jpg,1
1,../../../Dataset/img_celeba_cropped/021509.jpg,../../../Dataset/img_celeba_cropped/027436.jpg,0
2,../../../Dataset/img_celeba_cropped/044078.jpg,../../../Dataset/img_celeba_cropped/046414.jpg,1
3,../../../Dataset/img_celeba_cropped/044078.jpg,../../../Dataset/img_celeba_cropped/025159.jpg,0
4,../../../Dataset/img_celeba_cropped/018977.jpg,../../../Dataset/img_celeba_cropped/037049.jpg,1


### Loading pairs into TF Dataset

In [41]:
#dataset = tf.data.Dataset.from_tensor_slices((df['left'].values, df['right'].values, df['label'].values))

dataset_features = tf.data.Dataset.from_tensor_slices((df['left'].values, df['right'].values))

train_label_dataset = tf.data.Dataset.from_tensor_slices(df['label'].values)

In [42]:
def preprocessing(left, right):
        
    return convert_to_img(left), convert_to_img(right)


def convert_to_img(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img)
    #img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.expand_dims(img, axis=0)
    return img

def label_preprocessing(label):
    return tf.cast(label, tf.float32)

In [43]:
dataset_features = dataset_features.map(preprocessing)
train_label_dataset = train_label_dataset.map(label_preprocessing)


dataset = tf.data.Dataset.zip((dataset_features, train_label_dataset))

dataset.batch(batch_size, num_parallel_calls=AUTOTUNE)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [44]:
train_dataset = dataset.take(round(images_num * 0.8))
val_dataset = dataset.skip(round(images_num * 0.8))

### Input layers

In [30]:
left_input = layers.Input(shape=(224, 224, 3), name='left_input')
right_input = layers.Input(shape=(224, 224, 3), name='right_input')

### VGG16model

In [32]:
vgg=vgg16.VGG16(
    include_top=False,
    input_shape=image_shape,
    pooling='avg',
    weights='imagenet',
    classes=2)

for layer in vgg.layers[0:-1]:
    layer.trainable = False
    
for (i,layer) in enumerate(vgg.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)
    
left_vgg16 = vgg(left_input)
right_vgg16 = vgg(right_input)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False
19 GlobalAveragePooling2D True


### Merging resnet branches layers

In [33]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = tf.reduce_sum(tf.square(subtract(x, y)), axis=1, keepdims=True)
    return tf.sqrt(tf.maximum(sum_square, K.epsilon()))

In [34]:
merged = layers.subtract([left_vgg16, right_vgg16])
#merged = layers.Lambda(euclidean_distance, name="merge")([left_resnet, right_resnet])

flatten = layers.Flatten()(merged)

merged_dense_1 = layers.Dense(256, activation='relu')(flatten)
merged_dense_2 = layers.Dense(32, activation='relu')(merged_dense_1)

merged_output = layers.Dense(1, activation='sigmoid')(merged_dense_2)

model = Model(inputs = [left_input, right_input], outputs = merged_output)

In [35]:
# margin is a parametr settable by developer
def contrastive_loss_with_margin(margin):
    def contrastive_loss(y_true, y_pred):
        '''Contrastive loss from Hadsell-et-al.'06
        http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
        '''
        square_pred = tf.square(y_pred)
        margin_square = tf.square(tf.maximum(margin - y_pred, 0))
        return (y_true * square_pred + (1 - y_true) * margin_square)    
    return contrastive_loss

In [36]:
model.compile(optimizer=Adam(learning_rate=lr), loss=contrastive_loss_with_margin(margin=1.0), metrics=['accuracy'])

In [37]:
#plot_model(resnet, show_shapes=True)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 left_input (InputLayer)        [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 right_input (InputLayer)       [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 vgg16 (Functional)             (None, 512)          14714688    ['left_input[0][0]',             
                                                                  'right_input[0][0]']      

In [45]:
model_fit = model.fit(train_dataset, epochs=5, validation_data=val_dataset)

Epoch 1/5


ValueError: in user code:

    File "/Users/igor/miniforge3/envs/my_env/lib/python3.8/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/igor/miniforge3/envs/my_env/lib/python3.8/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/igor/miniforge3/envs/my_env/lib/python3.8/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/igor/miniforge3/envs/my_env/lib/python3.8/site-packages/keras/engine/training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/igor/miniforge3/envs/my_env/lib/python3.8/site-packages/keras/engine/training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "/Users/igor/miniforge3/envs/my_env/lib/python3.8/site-packages/ker

### Performance metrics

In [ ]:
plt.plot(combo_model_fitted.history['accuracy'])
plt.plot(combo_model_fitted.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.axis(ymin=0.5,ymax=1)
plt.grid()

plt.show()

In [ ]:
plt.plot(combo_model_fitted.history['loss'])
plt.plot(combo_model_fitted.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.axis(ymin=0,ymax=0.5)
plt.grid()

plt.show()

### Predictions

In [ ]:
left_image_feature_vectors = model.predict(custom_img_left)
left_image_feature_vectors = model.predict(custom_img_right)